In [ ]:
#imports

import pandas as pd
import numpy as np
from pandas import Series, DataFrame

import re

from os import listdir
from os.path import isfile, join
import gzip


In [ ]:
#DataFrame a partir de los zip de los subtitulos
def file_list(dir_path="./data/subtitles/"):
    
    #Listado de archivos en carpeta
    file_list = listdir(dir_path)
    return [join(dir_path, file_name) for file_name in file_list]

In [ ]:
#Función para pasar de rating numérico a categórico, el límite es arbitrario.
def ratingCategory(x):
    if x>=7:
        return 'Buy cinema ticket'
    else:
        return 'Wait for Netflix';

In [ ]:
#Levantamos la lista de pelis con los otros atributos.
def imdb_discretized_dataset(file_path = "./data/imdb_merged_movies.tsv"):
    movies_info=pd.read_csv(file_path,delimiter='\t',encoding='utf-8')
    #Discretización del rating
    movies_info['rating']=movies_info.averageRating.apply(lambda x: ratingCategory(x))
    return movies_info;

In [ ]:
def clean_text(text):
    
    #Elimino en nro inicial
    text = re.sub('(1\n)',"\n",text)
    
    #Eliminamos los tiempos
    text = re.sub('(\d{1,2}:\d{1,2}:\d{1,2},\d{1,3} --> \d{1,2}:\d{1,2}:\d{1,2},\d{1,3})',"",text)
    
    #Eliminamos los \n que andan sueltos por el texto
    text = re.sub('(\r{0,1}\n\d{0,4})'," ",text)
    #Eliminamos los \r que andan sueltos por el texto
    text = re.sub('(\r{0,1}\n)'," ",text)
    
    #Eliminamos los \' que están puestos para las abreviaturas, en vez de espacio 
    #debería reemplazarse por: "'" pero no me anduvo.
    text = re.sub("(\\')","",text)
    #Eliminamos los <i>...</i>
    text = re.sub("(<i>)","",text)
    text = re.sub("(</i>)","",text)
    
    #Eliminamos símbolo musical
    text = re.sub("♪","",text)
    #Eliminamos sentencias del estilo: \lalala , pero no me está funcionando, es porque en 
    #algunos textos vi: \ufeff1 o \xa0 
    #pero no sé por qué no me toma la regex. Igual el tokenizador lo saco, ver debajo.
    text = re.sub('(\\\w+\s)'," ",text)
    
    #Elimino todo lo que esté entre (), que son aclaraciones del contexto, no hacen al diálogo.
    #Por ejemplo ( onlookers groan ), ( man grunts, groans )
    #Le pongo el ? para que sea non-greedy.
    # Algunos están entre []; ejemplo [ jingle ]
    text = re.sub(r"\(.*?\)"," ", text)
    text = re.sub(r"\[.*?\]"," ", text)
    text = re.sub(r"\<.*?\>"," ", text)
    
    text = re.sub(r"\ufeff1"," ", text)
    #Elimino la publicidad del subtitutlo al final del documento
    text = re.sub("[cC]aption[s]{0,1} by.*$","", text)
    text = re.sub("[sS]ubtitle[s]{0,1} by.*$","", text)

    
    #Si hay varios espacios los dejamos en uno sólo.
    text = re.sub(" +"," ",text)
    
    text= text.lower().strip()
    return text 

In [ ]:
def get_text(file_path):
    # funcion simple que carga texto zipeado, lee y decodifica
    return clean_text(gzip.open(file_path).read().decode("utf8","ignore"))
    

In [ ]:
text_list = [get_text(file) for file in file_list()]
text_df = DataFrame({"tconst":[ f.split(".")[1].split("/")[-1] for f in file_list()],"raw_text":text_list})

#Join with IMDB dataset
#Obtengo la base de datos de IMDB, con la clase discretizada
imdb_df=imdb_discretized_dataset()

df = pd.merge(imdb_df,text_df, on="tconst" , how="inner").set_index("tconst")

In [ ]:
df.head()

In [ ]:
import pickle
#df.to_pickle('./data/dataframe/dataframe.pkl')
with open('./data/dataframe/dataframe.pkl', 'wb') as f:
        pickle.dump(df, f, protocol=2)

In [ ]:
my_movie=pd.read_pickle('./data/dataframe/dataframe.pkl')
len(my_movie)